**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


In [2]:
from IPython.display import Latex

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is to determine the next step whether to exploit the previous step or explore the new one. When it comes to why $epsilon$ is essential is that we use the $epsilon$ to decide the possibility of exploration.

To be detailed, in the python code we set a radom variable we can call it $p$ which is between 0 and 1, then we can compare $p$ with $epsilon$, is $p>epsilon$, we exploit the previous one, if $p<epsilon$, we explore the new one, a random one.

It is important to set the $epsilon$ value, we should take a balance when setting the $epsilon$, if too high, we will keep trying new random ones, if too low, we will not have many chances to explore. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=64 # set small when debugging
epochs_test=64 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Position:__<br>
The position represents the place in the island. It is set as $0$ initially, but if the rat moves to one of the position, the position will be $1$. This setting of position enables to reprent the movement of the rat. Also, there are $4$ places set to be $-1$ which reprents it is poisonous and the rat will not go.

__Board:__<br>
The board is used to represent the rat's visibility, which used by a size of $4$ in the code. Of the board, it also contains the rewards for the rat, $-1$ represents it is poisonous, and $0.5$ represents cheese. Also, it will exist $0$ if the rat already explored.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        win = 0
        lose = 0
        game_over = False
        # Start game
        state = env.reset()
        
        # Running the game
        while not game_over:
            action = agent.learned_act(state) 
            state, reward, game_over = env.act(action)
            
            if reward >= 0:
                win+=reward
            else:
                lose+=abs(reward)
                
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/11.0. Average score (-1.0)
Win/lose count 6.0/10.0. Average score (-2.5)
Win/lose count 9.5/13.0. Average score (-2.8333333333333335)
Win/lose count 12.0/16.0. Average score (-3.125)
Win/lose count 6.0/7.0. Average score (-2.7)
Win/lose count 11.5/16.0. Average score (-3.0)
Win/lose count 15.5/18.0. Average score (-2.9285714285714284)
Win/lose count 11.0/12.0. Average score (-2.6875)
Win/lose count 14.0/19.0. Average score (-2.9444444444444446)
Win/lose count 11.5/17.0. Average score (-3.2)
Win/lose count 12.5/18.0. Average score (-3.409090909090909)
Win/lose count 16.0/17.0. Average score (-3.2083333333333335)
Win/lose count 13.5/18.0. Average score (-3.3076923076923075)
Win/lose count 8.0/14.0. Average score (-3.5)
Win/lose count 10.0/15.0. Average score (-3.6)
Win/lose count 11.0/13.0. Average score (-3.5)
Win/lose count 11.5/15.0. Average score (-3.5)
Win/lose count 14.5/13.0. Average score (-3.2222222222222223)
Win/lose count 13.0/20.0. Average score (-3.421052

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Notice that:<br>
$\displaystyle Q^{\pi }( s,a) =E\left[ r_{t+1} +\lambda r_{t+2} +\lambda ^{2} r_{t+3} +...|s,a\right]$<br>
$\displaystyle Q^{\pi }( s,a) =E[ r_{t+1} +\lambda ( r_{t+2} +\lambda r_{t+3} +...) |s,a]$<br>
So that:<br>
$\displaystyle Q^{\pi }( s,a) =E_{s'}\left[ r+\lambda Q^{\pi}( s',a') |s,a\right]$<br>
i.e.<br>
$\displaystyle Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$<br>
Because $Q^{*}( s,a)$ is the optimal policy, so:<br>
$\displaystyle Q^{*}( s,a) =\underset{\pi }{\max} Q^{\pi }( s,a) =\max E_{s'}\left[ r+\lambda Q^{\pi }( s',a') |s,a\right]$<br>
In order to maximum $\displaystyle Q^{\pi }( s,a)$, we must firstly maximum $\displaystyle Q^{\pi }( s',a')$:<br>
$\displaystyle \underset{\pi }{\max} Q^{\pi }( s,a) =E_{s'}\left[ r+\lambda \underset{a'}{\max} Q^{\pi }( s',a') |s,a\right]$<br>
i.e.<br>
$ Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$<br>
<br>
But in order to get the loss function, we should further more, we notice that:<br>
$\displaystyle \underset{\pi '}{\max}\sum _{s'} Q^{\pi '}( s',a'|s,a) \leqslant \sum _{s',a'}\underset{\pi '}{\max} Q^{\pi '}( s',a'|s,a)$<br>
To get the optium solution, we should set a loss function like that:<br>
$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$<br>
After using this loss function, we can compare the Q-value with the inital equilty.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        i = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
from keras.layers import Dense, Activation, Flatten

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()          
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [27]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/064 | Loss 0.0060 | Win/lose count 2.5/6.0 (-3.5)
Epoch 001/064 | Loss 0.0071 | Win/lose count 4.5/4.0 (0.5)
Epoch 002/064 | Loss 0.0327 | Win/lose count 1.0/6.0 (-5.0)
Epoch 003/064 | Loss 0.0038 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/064 | Loss 0.0155 | Win/lose count 6.0/5.0 (1.0)
Epoch 005/064 | Loss 0.0092 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/064 | Loss 0.0669 | Win/lose count 4.5/4.0 (0.5)
Epoch 007/064 | Loss 0.0219 | Win/lose count 4.0/5.0 (-1.0)
Epoch 008/064 | Loss 0.0118 | Win/lose count 2.5/3.0 (-0.5)
Epoch 009/064 | Loss 0.0047 | Win/lose count 5.5/1.0 (4.5)
Epoch 010/064 | Loss 0.0084 | Win/lose count 1.0/2.0 (-1.0)
Epoch 011/064 | Loss 0.0190 | Win/lose count 2.5/1.0 (1.5)
Epoch 012/064 | Loss 0.0125 | Win/lose count 2.0/2.0 (0.0)
Epoch 013/064 | Loss 0.0125 | Win/lose count 6.5/6.0 (0.5)
Epoch 014/064 | Loss 0.0077 | Win/lose count 6.0/1.0 (5.0)
Epoch 015/064 | Loss 0.0584 | Win/lose count 4.0/1.0 (3.0)
Epoch 016/064 | Loss 0.0427 | Win/lose count 9.0/6

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, 2, activation='relu'))
        model.add(Conv2D(16, 2, activation='relu'))
        model.add(Flatten())
        model.add(Dense(16, activation='relu'))
        model.add(Dense(4, activation='softmax'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [29]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/064 | Loss 0.0065 | Win/lose count 0.5/2.0 (-1.5)
Epoch 001/064 | Loss 0.0079 | Win/lose count 1.0/2.0 (-1.0)
Epoch 002/064 | Loss 0.0034 | Win/lose count 5.0/2.0 (3.0)
Epoch 003/064 | Loss 0.0008 | Win/lose count 2.0/3.0 (-1.0)
Epoch 004/064 | Loss 0.0018 | Win/lose count 1.5/6.0 (-4.5)
Epoch 005/064 | Loss 0.0331 | Win/lose count 5.5/6.0 (-0.5)
Epoch 006/064 | Loss 0.0184 | Win/lose count 1.5/4.0 (-2.5)
Epoch 007/064 | Loss 0.0000 | Win/lose count 1.0/2.0 (-1.0)
Epoch 008/064 | Loss 0.0001 | Win/lose count 4.0/1.0 (3.0)
Epoch 009/064 | Loss 0.0022 | Win/lose count 3.0/7.0 (-4.0)
Epoch 010/064 | Loss 0.0095 | Win/lose count 3.5/5.0 (-1.5)
Epoch 011/064 | Loss 0.0001 | Win/lose count 3.5/1.0 (2.5)
Epoch 012/064 | Loss 0.0047 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/064 | Loss 0.0020 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/064 | Loss 0.0036 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/064 | Loss 0.0000 | Win/lose count 3.0/6.0 (-3.0)
Epoch 016/064 | Loss 0.0087 | Win/lose count 5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [32]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/0. Average score (5.0)
Win/lose count 1.0/1.0. Average score (2.5)
Win/lose count 4.0/0. Average score (3.0)
Win/lose count 5.5/1.0. Average score (3.375)
Win/lose count 1.0/0. Average score (2.9)
Win/lose count 4.5/0. Average score (3.1666666666666665)
Win/lose count 5.0/0. Average score (3.4285714285714284)
Win/lose count 6.0/1.0. Average score (3.625)
Win/lose count 11.0/0. Average score (4.444444444444445)
Win/lose count 8.0/0. Average score (4.8)
Win/lose count 3.0/0. Average score (4.636363636363637)
Win/lose count 10.0/0. Average score (5.083333333333333)
Win/lose count 6.0/1.0. Average score (5.076923076923077)
Win/lose count 2.5/0. Average score (4.892857142857143)
Win/lose count 1.5/0. Average score (4.666666666666667)
Win/lose count 8.0/0. Average score (4.875)
Win/lose count 6.5/0. Average score (4.970588235294118)
Win/lose count 1.5/0. Average score (4.777777777777778)
Win/lose count 1.5/0. Average score (4.605263157894737)
Win/lose count

In [36]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 17.0/0. Average score (17.0)
Win/lose count 3.5/0. Average score (10.25)
Win/lose count 11.5/1.0. Average score (10.333333333333334)
Win/lose count 8.0/0. Average score (9.75)
Win/lose count 4.5/0. Average score (8.7)
Win/lose count 6.0/0. Average score (8.25)
Win/lose count 12.0/0. Average score (8.785714285714286)
Win/lose count 11.0/0. Average score (9.0625)
Win/lose count 6.0/1.0. Average score (8.61111111111111)
Win/lose count 5.5/1.0. Average score (8.2)
Win/lose count 3.5/0. Average score (7.7727272727272725)
Win/lose count 11.0/1.0. Average score (7.958333333333333)
Win/lose count 19.0/0. Average score (8.807692307692308)
Win/lose count 9.5/0. Average score (8.857142857142858)
Win/lose count 0.5/0. Average score (8.3)
Win/lose count 3.0/0. Average score (7.96875)
Win/lose count 5.5/0. Average score (7.823529411764706)
Win/lose count 1.5/0. Average score (7.472222222222222)
Win/lose count 0.5/0. Average score (7.105263157894737)
Win/lose count 3.5/

In [35]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/1.0. Average score (2.0)
Win/lose count 4.0/0. Average score (3.0)
Win/lose count 13.0/0. Average score (6.333333333333333)
Win/lose count 6.0/0. Average score (6.25)
Win/lose count 35.0/1.0. Average score (11.8)
Win/lose count 7.5/0. Average score (11.083333333333334)
Win/lose count 15.5/0. Average score (11.714285714285714)
Win/lose count 19.0/1.0. Average score (12.5)
Win/lose count 21.0/0. Average score (13.444444444444445)
Win/lose count 6.0/0. Average score (12.7)
Win/lose count 36.0/1.0. Average score (14.727272727272727)
Win/lose count 11.0/2.0. Average score (14.25)
Win/lose count 23.0/1.0. Average score (14.846153846153847)
Win/lose count 14.5/0. Average score (14.821428571428571)
Win/lose count 18.0/0. Average score (15.033333333333333)
Win/lose count 7.0/0. Average score (14.53125)
Win/lose count 1.5/0. Average score (13.764705882352942)
Win/lose count 14.0/0. Average score (13.777777777777779)
Win/lose count 8.0/0. Average score (13.47368

In [33]:
HTML(display_videos('cnn_test10.mp4'))

In [34]:
HTML(display_videos('fc_test10.mp4'))

Obviously, the CNN algorithm is better than FC algorithm a lot, according to the changes of temperatures we can notice that. But the most important issue is that because of the setting of epsilon in the previous code, the algorithms are tending to keep the initial socre, which results from not exploring new states and only exploiting the initial ones. Because of not exploring, we can observe that the rewards are little, and the final socre keeps close to the initial value, this issue is sepcially obvious when the values of temparatures are small.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [21]:
def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    score = 0
    loss = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
    
    def act(self, action,train=False):
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    def reset(self):

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [22]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/064 | Loss 0.0060 | Win/lose count 5.5/28.900000000000112 (-23.400000000000112)
Epoch 001/064 | Loss 0.0100 | Win/lose count 1.5/20.10000000000002 (-18.60000000000002)
Epoch 002/064 | Loss 0.0057 | Win/lose count 3.0/23.200000000000063 (-20.200000000000063)
Epoch 003/064 | Loss 0.0134 | Win/lose count 3.0/21.400000000000045 (-18.400000000000045)
Epoch 004/064 | Loss 0.0151 | Win/lose count 5.5/20.600000000000026 (-15.100000000000026)
Epoch 005/064 | Loss 0.0075 | Win/lose count 4.5/22.200000000000053 (-17.700000000000053)
Epoch 006/064 | Loss 0.0175 | Win/lose count 1.0/21.70000000000004 (-20.70000000000004)
Epoch 007/064 | Loss 0.0209 | Win/lose count 1.5/20.20000000000002 (-18.70000000000002)
Epoch 008/064 | Loss 0.0042 | Win/lose count 1.0/21.600000000000044 (-20.600000000000044)
Epoch 009/064 | Loss 0.0040 | Win/lose count 1.5/21.500000000000036 (-20.000000000000036)
Epoch 010/064 | Loss 0.0117 | Win/lose count 3.0/22.500000000000057 (-19.500000000000057)
Epoch 011/064 | 

In [23]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 0.5/0. Average score (0.5)
Win/lose count 1.0/4.0. Average score (-1.25)
Win/lose count 0.5/1.0. Average score (-1.0)
Win/lose count 0.5/2.0. Average score (-1.125)
Win/lose count 0.5/1.0. Average score (-1.0)
Win/lose count 2.0/1.0. Average score (-0.6666666666666666)
Win/lose count 0.5/0. Average score (-0.5)
Win/lose count 1.0/0. Average score (-0.3125)
Win/lose count 0.5/0. Average score (-0.2222222222222222)
Win/lose count 0.5/0. Average score (-0.15)
Win/lose count 1.5/4.0. Average score (-0.36363636363636365)
Win/lose count 3.0/0. Average score (-0.08333333333333333)
Win/lose count 1.5/1.0. Average score (-0.038461538461538464)
Win/lose count 2.5/3.0. Average score (-0.07142857142857142)
Win/lose count 1.5/2.0. Average score (-0.1)
Win/lose count 0.0/0. Average score (-0.09375)
Win/lose count 2.0/3.0. Average score (-0.14705882352941177)
Win/lose count 1.5/0. Average score (-0.05555555555555555)
Win/lose count 1.5/1.0. Average score (-0.02631578947368421)
Win/lose

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***